In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
img = cv.imread('Images/Echantillion1Mod2_301.png', cv.IMREAD_GRAYSCALE)
img_RGB = cv.cvtColor(cv.imread('Images/Echantillion1Mod2_301.png'), cv.COLOR_BGR2RGB)

In [ ]:
def show_image(img, title, cmap='gray'):
    plt.figure(figsize=(10, 10))
    plt.imshow(img, cmap=cmap)
    plt.axis('off')
    plt.title(title)
    plt.show()

In [ ]:
show_image(img, 'grayscale image')

In [ ]:
blur_img = cv.GaussianBlur(img, (5, 5), 0)
thresh_img = cv.threshold(blur_img, 40, 255, cv.THRESH_BINARY)[1]
show_image(thresh_img, 'threshold image')

In [ ]:
open_img = cv.morphologyEx(thresh_img, cv.MORPH_OPEN, np.ones((5, 5)))
show_image(open_img, 'image after an openning')

In [ ]:
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

In [ ]:
distance = ndi.distance_transform_edt(open_img)
local_maxi = peak_local_max(distance, footprint=np.ones((55, 55)), labels=open_img)
peaks = np.zeros_like(open_img)
peaks[tuple(local_maxi.T)] = 255
# show_image(peaks, 'peaks')

In [ ]:
markers = ndi.label(peaks)[0]
plt.colorbar(plt.imshow(markers, cmap='magma'))
plt.axis('off')
plt.show()

In [ ]:
labels = watershed(-distance, markers, mask=open_img)
colored_rocks = np.zeros_like(img_RGB)
for i in range(1, labels.max() + 1):
    colored_rocks[labels == i] = np.random.randint(0, 255, 3)
fig, (og_img, segmented_img) = plt.subplots(1, 2, figsize=(20, 10))
og_img.imshow(img_RGB)
og_img.axis('off')
og_img.set_title('Original image')
segmented_img.imshow(colored_rocks)
segmented_img.axis('off')
segmented_img.set_title('Segmented image')
plt.show()